In [55]:
#!pip install transformers
#!pip install jinja2 --upgrade

In [56]:
#from huggingface_hub import notebook_login
#notebook_login()

In [1]:
# Load mention data
from pynif import NIFCollection

nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif"

nif_data = ""

with open(nif_data_path, 'r', encoding="utf-8") as f:
    nif_data = f.read()

parsed_collection = NIFCollection.loads(nif_data, format='turtle')

In [2]:
for context in parsed_collection.contexts:
    # See what can be called on context
    # print(dir(context))

    # input sentence (1 document = 1 sentence)
    document = context.mention
    print(f"{context.mention}, {context.beginIndex}, {context.endIndex}")
    for phrase in context.phrases:
        #print(phrase)
        #print(dir(phrase))
        print(f"{phrase.mention}, {phrase.beginIndex}, {phrase.endIndex}, {phrase.generated_uri}, {phrase.taClassRef}, {phrase.taIdentRef}")
    break

Stork H1 results breakdown per sector .  AMSTERDAM 1996-08-28  First 24 weeks 1996  ( millions of guilders unless otherwise stated )  Industrial systems and components  - Turnover 756 vs 829  - Operating profit 46 vs 48  - New orders received 876 vs 933  - Order book ( billions ) 1.07 vs 0.98  Industrial services  - Turnover 657 vs 700  - Operating profit 9 vs 3  - New orders received ( billions ) 1.00 vs 1.09  - Order book ( billions ) 2.37 vs 2.01  NOTE - Order book figures refer to value of orders on books at end of period .  -- Amsterdam newsroom +31 20 504 5000 , Fax +31 20 504 5040 , 0, 595
AMSTERDAM, 41, 50, https://aifb.kit.edu/conll/768#offset_41_50, None, http://en.wikipedia.org/wiki/Amsterdam
Amsterdam, 538, 547, https://aifb.kit.edu/conll/768#offset_538_547, None, http://en.wikipedia.org/wiki/Amsterdam


In [205]:
def transform_dbp_json_cands_to_dict(json_documents={}):
    candidate_dict = {}
    candidate_dict['types'] = []
    candidate_dict['label'] = []
    candidate_dict['desc'] = []
    candidate_dict['uri'] = []
    candidate_dict['score'] = []
    candidate_dict['refCount'] = []
    
    for i in range(len(json_documents)):
        document = json_documents[i]
        try:
            # Retrieve information
            # Description
            small_desc = document.get('comment', "")
            # Types for this candidate
            types = document.get('typeName', [])
            if types is None:
                #print(f"No types for: {document.keys()}, {document['label']}")
                types = ""
                
            # The label / name for this candidate
            label = document.get('label', [])
            
            # URI for this candidate
            uri = document.get('resource', [])

            # Score (Optional)
            score = document.get('score', [])

            # Reference Count (Optional)
            refCount = document.get('refCount', [])

            # Populate information
            candidate_dict['types'].append(types)
            candidate_dict['label'].extend(label)
            candidate_dict['desc'].extend(small_desc)
            candidate_dict['uri'].extend(uri)
            candidate_dict['score'].extend(score)
            candidate_dict['refCount'].extend(refCount)

            
        except:
            print(f"ERROR: {document.keys()}")
            raise
    return candidate_dict['label'], candidate_dict['desc'], candidate_dict['types'], candidate_dict['uri'], candidate_dict['score'], candidate_dict['refCount']


labels, desc, types, uris, score, refCount = transform_dbp_json_cands_to_dict(json_documents)

In [4]:
import json
from os import listdir
from os.path import isfile, join
        
def load_candidate_results(
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/"):
    all_candidate_results = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            
            all_candidate_results[path] = in_json
    return all_candidate_results

In [2]:
# Load mention data
from pynif import NIFCollection

def load_nif_dataset(nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif"):

    nif_data = ""

    with open(nif_data_path, 'r', encoding="utf-8") as f:
        nif_data = f.read()

    parsed_collection = NIFCollection.loads(nif_data, format='turtle')

parsed_collection = load_nif_dataset(nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif")

In [5]:
all_candidate_results = load_candidate_results(result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/")
print(f"Number of entries: {len(all_candidate_results)}")

Number of entries: 1393


In [6]:
#get some random key to get one entry
key = (list(all_candidate_results.keys()))[10]
#print(all_candidate_results[list(all_candidate_results.keys())[10]])
print(key)
#key = next(iter(all_candidate_results))
candidates_entry = all_candidate_results[key]
# Only has one key
in_text = next(iter( candidates_entry ))
json_candidates = candidates_entry[in_text]
json_candidates.keys()

C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/1006.json


dict_keys(['mentions', 'begin_index', 'end_index', 'labels', 'desc', 'types', 'candidate_uris', 'scores', 'refCount', 'true_uris'])

In [19]:
def load_cand_desc_results(
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidate_descriptions/CoNLL_AIDA-YAGO2-dataset.nif/"):
    '''Taken from query_data.ipynb'''
    all_cand_desc_results = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            
            all_cand_desc_results[path] = in_json
    return all_cand_desc_results




In [20]:

def load_dict_candidate_description():
    all_cand_desc_results = load_cand_desc_results()
    # uris_for_descriptions --> all URIs we want to crawl
    # cands_desc

    dict_uri_desc = {}
    for filename in all_cand_desc_results:
        cands_desc = all_cand_desc_results[filename]
        #print(list(cands_desc.keys())[0])
        for uri in cands_desc:
            desc = cands_desc[uri]
            dict_uri_desc[uri] = desc
    return dict_uri_desc



In [27]:
# Save it globally to avoid long call chains...
dict_uri_description = load_dict_candidate_description()


def get_or_default_lst(lst, idx, default):
    return lst[idx] if idx < len(lst) else default


def generate_prompt_options(json_candidates, i, newline='\n'):
    # i-th mention
    # j-th candidate
    prompt_options = []
    prompt_options_w_desc = []
    candidate_uris = []
    for j in range(len(json_candidates)):
        mention = json_candidates['mentions'][i]
        # Types for this one candidate
        types_list = get_or_default_lst(json_candidates['types'], i, [])
        types_list = get_or_default_lst(types_list, j, [])
        types = ', '.join(types_list)
        
        # this candidate's URI
        uri_list = get_or_default_lst(json_candidates['candidate_uris'], i, [])
        uri = get_or_default_lst(uri_list, j, "")
        
        # Description for this candidate
        desc = dict_uri_description.get(uri, "")

        #desc_list = get_or_default_lst(json_candidates['desc'], i, [])
        #desc = get_or_default_lst(desc_list, j, "")
        
        prompt_choice = f"{j}. { mention } ({ types }) - { uri }"
        prompt_choice_w_desc = prompt_choice + f" {newline}{ desc }"
        prompt_options_w_desc.append(prompt_choice_w_desc)
        prompt_options.append(prompt_choice)
        candidate_uris.append(uri)
    
    return prompt_options_w_desc, prompt_options, candidate_uris


def generate_prompt_from_candidates(json_candidates, input_document):
    newline = "\n"
    mention_prompts = []
    mention_prompt_choices = []
    mention_candidate_uris = []
    
    #input_document, mentions, labels, candidate_uris
    for mention_index in range(len(json_candidates['mentions'])):
        prompt_options_w_desc, prompt_choices, candidate_uris = generate_prompt_options(json_candidates=json_candidates, i=mention_index)
        mention_prompt_choices.append(prompt_choices)
        mention_candidate_uris.append(candidate_uris)
        prompt_text = f"""For the following input document, which of the listed entity candidates (if any) is referred to by mention \"{json_candidates['mentions'][mention_index]}\" (at offset {json_candidates['begin_index'][mention_index]})? 
Please reply with just the index and the URI for the entity.
Input document:
\"{input_document}\" 

Entity Candidates:
{(newline+newline).join(prompt_options_w_desc)}"""
        
        mention_prompts.append(prompt_text)
    return mention_prompts, json_candidates['true_uris'], mention_prompt_choices, mention_candidate_uris
        



In [28]:
mention_prompts, true_uris, mention_prompt_choices, mention_candidate_uris = generate_prompt_from_candidates(json_candidates, input_document=in_text)

In [29]:
mention_prompts[0]

'For the following input document, which of the listed entity candidates (if any) is referred to by mention "Arab" (at offset 1504)? \nInput document:\n        "Main Tunisian opposition party ousted from HQ .  TUNIS 1996-08-24  Tunisia \'s main opposition party on Saturday announced that it had been ousted from its headquarters building by a court decision for failing to pay the rent .  Mohamed Ali Khalfallah , spokesman for the Movement of Socialist Democrats ( MDS ) said that a bailiff who was accompagnied by policemen , on Saturday ordered the party to leave the building .  " We were not allowed a delay to enable us to transfer the movement \'s goods and documents , " Khalfallah added in a statement .  The building is state property .  The MDS was represented in court and admitted owing money for rent but did not give details .  MDS this year lost its president and vice-president , both of whom were tried and given jail sentences .  MDS president Mohamed Moada was sentenced last Feb

In [24]:
# Fuzzy string matching library
#!pip install thefuzz

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.0 MB/s eta 0:00:00


In [23]:
def find_matching_choice(response: str, choices: list):
    # Return best rated index for passed list
    max_ratio = 0
    max_ratio_idx = 0
    response = response.lower()
    #print(f"To match:{response}")
    for choice_idx in range(len(choices)):
        choice = choices[choice_idx].lower()
        
        if len(response) < len(choice):
            shorter = response
            longer = choice
        else:
            shorter = choice
            longer = response
            
        new_ratio = fuzz.partial_ratio(shorter, longer)
        #print(f"{choice}: {new_ratio}")
        if new_ratio > max_ratio:
            max_ratio = new_ratio
            max_ratio_idx = choice_idx
        if max_ratio >= 100:
            # Quick closure on exact matches...
            return max_ratio_idx
    return max_ratio_idx

In [43]:
from transformers import AutoTokenizer
from huggingface_hub import InferenceClient
from tqdm import tqdm_notebook

# Fuzzy matching for LLM responses
from thefuzz import fuzz


def query_LLM(input_document="", json_candidates={}, endpoint_url="http://aifb-websci-gpunode2.aifb.kit.edu:8081", \
              model_id="meta-llama/Meta-Llama-3-8B-Instruct"):

    tokenizer=AutoTokenizer.from_pretrained(model_id)
    y_pred_dbp = []
    y_true = []

    # Generate prompts for what we want
    mention_prompts, true_uris, mention_prompt_choices, mention_candidate_uris = generate_prompt_from_candidates(json_candidates, input_document=input_document)

    client = InferenceClient(endpoint_url)

    # One prompt per mention is executed
    for i in tqdm_notebook(range(len(mention_prompts))):
        #print(f"Query Numero #{i}")
        messages = [
            {"role": "system", "content": "You are an expert assistant disambiguating entities and outputting if any of the passed entities are referenced in a given input text."},
            #{"role": "user", "content": f"Please convert this text into triples: 'Green Day is a rock band.'"},            
            #{"role": "assistant", "content": "Here are the triples corresponding to 'Green Day is a rock band.': (Green Day, has type, rock band)"},
            #{"role": "user", "content": f"Please convert this text into triples: 'New York is a city in North America.'"},            
            #{"role": "assistant", "content": "Here are the triples corresponding to 'New York is a city in North America': (New York, has type, city), (New York, located in, North America)"},
            {"role": "user", "content": prompt_text_apple},
            {"role": "assistant", "content": "The correct disambiguated entity is number 2. Apple Inc. (Public company) - http://dbpedia.org/resource/Apple_Inc."},
            #{"role": "user", "content": prompt_text_steve},
            #{"role": "assistant", "content": "The correct disambiguated entity is number 11. Steve Jobs (animal) - http://dbpedia.org/resource/Steve_Jobs"},
            {"role": "user", "content": mention_prompts[i]},
            {"role": "assistant", "content": "The correct disambiguated entity is number"},
        ]

        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )


        response = client.text_generation(
          prompt + f"The correct disambiguated entity is number",
          max_new_tokens=64,
          do_sample=False,
          temperature=0.01,
                stop_sequences=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"]
        )

        #print(prompt + response)
        #
        #print(f"Wanted entity: {true_uris[i]}")
        #print(f"RESPONSE is: {response}")
        choice_index = find_matching_choice(response, mention_prompt_choices[i])
        #print(f"Candidate Option #{choice_index}: {mention_prompt_choices[i][choice_index]} was chosen.")
        # See if given answer matches the true label
        chosen_candidate = mention_candidate_uris[i][choice_index]
        wanted_candidate = true_uris[i]
        #print(f"Chosen[{chosen_candidate}] vs. wanted[{wanted_candidate}]")
        y_true.append(wanted_candidate)
        y_pred_dbp.append(chosen_candidate)
    
    return y_true, y_pred_dbp


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_results(y_pred, y_true, average='macro'):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    #print("Accuracy:", accuracy)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average=average)
    #print("Precision:", precision)

    # Calculate recall (sensitivity)
    recall = recall_score(y_true, y_pred, average=average)
    #print("Recall (Sensitivity):", recall)

    # Calculate F1-score
    f1 = f1_score(y_true, y_pred, average=average)
    #print("F1-Score:", f1)
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [8]:
import os
from os import listdir
from os.path import isfile, join
import json

def persist_prompt_and_results(output_directory, in_filepath, input_document, mention_prompts, y_true, y_pred, mention_prompt_choices, mention_candidate_uris, eval_results):
    # Build a great json!
    prompt_dict = {}
    prompt_dict['cand path'] = in_filepath
    prompt_dict['doc'] = input_document
    prompt_dict['mention_prompts'] = mention_prompts
    prompt_dict['y_true'] = y_true
    prompt_dict['y_pred'] = y_pred
    prompt_dict['mention_prompt_choices'] = mention_prompt_choices
    prompt_dict['mention_candidate_uris'] = mention_candidate_uris
    prompt_dict['eval_results'] = eval_results
    
    
    with open(output_directory+str(os.path.basename(in_filepath)), 'w', encoding='utf-8') as f:
        json.dump(prompt_dict, f, ensure_ascii=False, indent=4)

        
def load_prompt_and_results(result_directory):
    prompt_dicts = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            prompt_dict = json.load(json_file)
            prompt_dicts[path] = prompt_dict
    return prompt_dicts
    

In [54]:
# All in one cell
from tqdm import tqdm_notebook

def load_dataset_queryLLM_evaluate(nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif", \
                                   cand_result_directory = "C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/",
                                  output_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/disambiguation/"):
    
    #print("[Loading NIF dataset] Started.")
    #dataset = load_nif_dataset(nif_data_path= nif_data_path)
    #print("[Loading NIF dataset] Completed.")
    print("[Loading candidate results] Started.")
    all_candidate_results = load_candidate_results(result_directory= cand_result_directory)
    print("[Loading candidate results] Completed.")
    print(f"Number of entries: {len(all_candidate_results)}")


    # Iterate through all entries
    for key in tqdm_notebook(all_candidate_results, position=0):
        #get some random key to get one entry
        #key = (list(all_candidate_results.keys()))[10]
        #print(all_candidate_results[list(all_candidate_results.keys())[10]])
        
        #print(key)#file name of which candidates were loaded from
        #key = next(iter(all_candidate_results))
        candidates_entry = all_candidate_results[key]
        # Only has one key
        input_document = next(iter( candidates_entry ))

        json_candidates = candidates_entry[input_document]

        # Generate prompts and get the possible choices, candidates etc. for choice matching and evaluation
        mention_prompts, true_uris, mention_prompt_choices, mention_candidate_uris = generate_prompt_from_candidates(json_candidates, input_document=input_document)

        #query_LLM
        #print(input_document)
        y_true, y_pred = query_LLM(input_document, json_candidates, endpoint_url = "http://aifb-websci-gpunode2.aifb.kit.edu:8081", model_id = "meta-llama/Meta-Llama-3-8B-Instruct")

        # Evaluate this document
        y_pred_wiki = [y.replace('http://dbpedia.org/resource/', 'http://en.wikipedia.org/wiki/') for y in y_pred]

        eval_results = evaluate_results(y_pred_wiki, y_true)
        
        # Save all results to files
        persist_prompt_and_results(output_directory, key, input_document, mention_prompts, y_true, y_pred, mention_prompt_choices, mention_candidate_uris, eval_results)

In [ ]:
load_dataset_queryLLM_evaluate()

In [9]:
prompt_result_dicts = load_prompt_and_results(result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/disambiguation/")

In [ ]:
all_true = []
all_pred = []
scored_results = []

for key in prompt_result_dicts.keys():
    y_true = prompt_result_dicts[key]['y_true']
    y_pred = prompt_result_dicts[key]['y_pred']
    y_pred_wiki = [y.replace('http://dbpedia.org/resource/', 'http://en.wikipedia.org/wiki/') for y in y_pred]
    if len(y_true) != len(y_pred):
        print(f"Error with lengths... {key}")
        print(prompt_result_dicts[key])
        raise ValueError(f"True:{ y_true } vs. Pred:{ y_pred }")
    all_true.extend(y_true)
    all_pred.extend(y_pred_wiki)
    eval_results = evaluate_results(y_pred_wiki, y_true, 'macro')
    scored_results.append((eval_results['f1'], key))


print(evaluate_results(all_pred, all_true, 'macro'))

In [21]:
scored_results.sort(key=lambda x: x[0])

In [ ]:
scored_results[0:20]

In [30]:
# Let's test some baselines
test_question = "What Wikipedia entity does the first occurrence of Ethiopia refer to in the following text?\n"
test_in_text = "SOCCER - AFRICAN NATIONS CUP COLLATED RESULTS .  JOHANNESBURG 1996-08-26  Collated results of African Nations Cup preliminary round , second leg matches played at the weekend :  Ethiopia 1 Uganda 1  2-2 on aggregate .  Ethiopia win 4-2 on penalties  Mauritania v Benin postponed to Friday  Benin lead 4-1 from the first leg  Namibia 6 Botswana 0  Namibia win 6-0 on aggregate  Seychelles 1 Mauritius 1  Mauritius win 2-1 on aggregate  Togo 1 Congo 0  Togo win 1-0 on aggregaete  Central African Republic walkover v Burundi  Winners progress to qualifying groups to start in October ."
test_prompt = test_question + "\n" + test_in_text

test_question_which = "Which of the following candidates fits the description of "
test_candidates = "\n0. Ethiopia (Place, Country, PopulatedPlace, Location) - http://dbpedia.org/resource/Ethiopia \n<B>Ethiopia</B> (; Amharic: áŠ¢á‰µá‹®áŒµá‹«, Ê¾ÄªtyÅ�á¹—á¹—yÄ�, , Tigrinya: áŠ¢á‰µá‹®áŒµá‹«, Oromo: Itoophiyaa, Somali: Itoobiya, Afar\n\n1. Ethiopia () - http://dbpedia.org/resource/Districts_of_Ethiopia \n divisions of <B>Ethiopia</B>. They are further subdivided into a number of wards (kebele) or neighbourhood\n\n2. Ethiopia (Place, Country, PopulatedPlace, Location) - http://dbpedia.org/resource/Ethiopian_Empire \n <B>Ethiopia</B> (; Ge'ez: áŠ¢á‰µá‹®áŒµá‹«, Amharic: áŠ¢á‰µá‹®áŒµá‹«, Ê¾ÄªtyÅ�á¹—á¹—yÄ�, , Tigrinya: áŠ¢á‰µá‹®áŒµá‹«, Oromo: Itoophiyaa, Somali\n\n3. Ethiopia (AdministrativeRegion, Place, PopulatedPlace, Location, Region) - http://dbpedia.org/resource/Amhara_Region \n of <B>Ethiopia</B>, containing the homeland of the Amhara people. Previously known as \"Region 3\", its capital\n\n4. Ethiopia () - http://dbpedia.org/resource/Ethiopia–United_States_relations \n<B>Ethiopia</B>–United States relations are bilateral relations between <B>Ethiopia</B> and the United States\n\n5. Ethiopia (Organisation, SoccerClub, Agent, SportsClub) - http://dbpedia.org/resource/Ethiopia_national_football_team \nThe <B>Ethiopia</B> national football team, nicknamed Walias, after the Walia ibex, represents <B>Ethiopia</B>\n\n6. Ethiopia (Settlement, City, Place, PopulatedPlace, Location) - http://dbpedia.org/resource/Addis_Ababa \n of <B>Ethiopia</B>. According to the 2007 census, the city has a population of 2,739,551 inhabitants. The city\n\n7. Ethiopia () - http://dbpedia.org/resource/Regions_of_Ethiopia \n<B>Ethiopia</B> is a federation subdivided into ethno-linguistically based Regional States (plural\n\n8. Ethiopia () - http://dbpedia.org/resource/Emperor_of_Ethiopia \nThe Emperor of <B>Ethiopia</B> (Ge'ez: áŠ•áŒ‰áˆ  áŠ�áŒˆáˆ¥á‰µ, nÉ™gusÃ¤ nÃ¤gÃ¤st, \"King of Kings\") was the hereditary ruler\n\n9. Ethiopia () - http://dbpedia.org/resource/Demographics_of_Ethiopia \nThe demographics of <B>Ethiopia</B> encompass the demographic features of <B>Ethiopia's</B> inhabitants"

In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import InferenceClient

def prompt_LLM(messages = []):

    endpoint_url = "http://aifb-websci-gpunode2.aifb.kit.edu:8081"
    model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    client = InferenceClient(endpoint_url)

    prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
    )


    response = client.text_generation(
      prompt,# + f"The correct disambiguated entity is number",
      max_new_tokens=64,
      do_sample=False,
      temperature=0.01,
            stop_sequences=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"]
    )
    return response


first_round_messages = [
    {"role": "system", "content": "You are a very well paid expert assistant disambiguating Wikipedia entities from texts."},
    #{"role": "user", "content": f"Please convert this text into triples: 'Green Day is a rock band.'"},            
    #{"role": "assistant", "content": "Here are the triples corresponding to 'Green Day is a rock band.': (Green Day, has type, rock band)"},
    #{"role": "user", "content": f"Please convert this text into triples: 'New York is a city in North America.'"},            
    #{"role": "assistant", "content": "Here are the triples corresponding to 'New York is a city in North America': (New York, has type, city), (New York, located in, North America)"},
    #{"role": "user", "content": f"Please convert this text into triples: 'Kris is a researcher'"},
    #
    #{"role": "user", "content": prompt_text_apple},
    #{"role": "assistant", "content": "The correct disambiguated entity is number 2. Apple Inc. (Public company) - http://dbpedia.org/resource/Apple_Inc."},
    #{"role": "user", "content": prompt_text_steve},
    #{"role": "assistant", "content": "The correct disambiguated entity Steve Jobs - http://wikipedia.org/page/Steve_Jobs"},
    {"role": "user", "content": test_prompt},
    {"role": "assistant", "content": "The first occurrence of \"Ethiopia\" in the text refers to the"},
    
]

second_round_messages = first_round_messages[0:-2]
second_round_messages.extend([{"role": "user", "content": test_prompt_which}, ])


entity_desc = prompt_LLM(first_round_messages)

test_prompt_which = test_question_which + entity_desc + "\n" + test_candidates

print(response)




In [ ]:
print(prompt_LLM([{"role": "user", "content": """Given the sentence \"Steve eats an apple,\" select the appropriate Wikipedia entity for the term \"apple\" from the following options:

1. Apple Inc. - A multinational technology company that designs, manufactures, and sells consumer electronics, software, and online services. Known for products like the iPhone, iPad, and MacBook.

2. Apple (Name) - A given name or surname. Sometimes used as a personal name.

3. Apple (Fruit) - The edible fruit produced by an apple tree (Malus domestica). Commonly eaten raw or used in cooking and baking.

4. Apple (Symbolism) - A symbol used in various cultural, religious, and artistic contexts. Often represents knowledge, immortality, temptation, or sin."""}]))

TODOs for evaluation:</br>
* sameAs links</br>
* is dbo:wikiPageRedirects of</br>
        * Example: http://en.wikipedia.org/wiki/People's_Republic_of_China vs. http://dbpedia.org/resource/China</br>
        * Relevant file: C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/1.json</br>
    * dbr:People's_Republic_Of_China</br>
